### Using LLama3

https://python.langchain.com/docs/integrations/llms/llamacpp/


This notebook requires a llama server to run in another terminal.

To install the server:

python -m venv venv

pip install --upgrade --quiet  llama-cpp-python
pip install sse_starlette
pip install starlette_context
pip install pydantic_settings
pip install openai


Download a Llama model from huggingface, e.g.:

 https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF

and place it in a local folder.


To launch llama_server in another terminal specify the path to the local model and the port:

    python -m llama_cpp.server --host 0.0.0.0 --model ./models/Meta-Llama-3-8B-Instruct.Q2_K.gguf --n_ctx 2048 --port 9001

This notebook creates an OPenAI client and will send the prompt request to the server for a response

In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from openai import OpenAI

https://platform.openai.com/examples

In [2]:
client = OpenAI(base_url="http://localhost:9001/v1", api_key="not-needed")

In [3]:
# Path to the JSON file
file_path = './clean_conversations/self-care_clean.json'

# Open and read the file
with open(file_path, 'r') as file:
    data = json.load(file)

# Now data is a dictionary or list depending on the JSON structure
print(data.keys())


dict_keys(['self-care'])


In [4]:
str_l = list(data.values())
print(type(str_l[0]))

<class 'list'>


In [5]:
sent_e=str_l[0][1]
sent_e

"F: How has your day been so far in terms of your daily routine?\nP: Well, I usually start the day with showering soon after waking up in the morning. \nF: Oh, that sounds refreshing! Do you prefer a quick shower? \nP: Yes, a quick shower to wake me up and get ready for the day. \nF: Do you often wash your hands throughout the day as well? \nP: Definitely, especially before handling food and after using the restroom. \nF: That's important for staying healthy. What about washing your face? \nP: I usually wash my face twice a day – once in the morning and once at night. \nF: It's good to keep a simple routine!"

In [6]:
import re

text = sent_e

# delete the question if it's the last sentence
if re.search(r'\nF:.*$', text):
    p = r'\nF:.*$'
    new_text = re.sub(p, '', text)
elif re.search(r'\nP:.*$', text):
    new_text = text
else:
    new_text = text

print("Modified text:", new_text)


Modified text: F: How has your day been so far in terms of your daily routine?
P: Well, I usually start the day with showering soon after waking up in the morning. 
F: Oh, that sounds refreshing! Do you prefer a quick shower? 
P: Yes, a quick shower to wake me up and get ready for the day. 
F: Do you often wash your hands throughout the day as well? 
P: Definitely, especially before handling food and after using the restroom. 
F: That's important for staying healthy. What about washing your face? 
P: I usually wash my face twice a day – once in the morning and once at night. 


## prompting engineering - zero-shot

### 1st try
instruct: almost orginal prompting

result :there is activity diescription (like the off-line description) and there is a note to record why the model asks this way


In [7]:

# system plays the role of care-takers
instruct_assis = {"role": "system", "content": "You are a care taker that has a conversation with an elderly person. \
             I will give you the conversation history in which the elderly person describes an activity and you give a response. \
             Answer with follow-up questions to ask about more specific questions to monitor the self care level.\
             The follow-up questions can be about more specific questions to reflect the functioning. \
             For example, when the activity is washing body parts, the follow up questions can be ask about the performance difficulty of the activity.\
             The follow-up questions can also ask the person's perception or feeling about the activity."

           }

instruct_user = {"role": "system", "content": "You are a an elderly person that has a conversation with a care taker. \
             You are asked with questions and you always provide well-reasoned answers to the questions that are both natural and short."

           }


# .(none, absent, negligible,... ) such as mild, moderate, severe and complete performance


## 2nd try
instruct: 
1. general: speicify this is online conversation to avoid the description of the motivation of the ct
2. (zero-shot) continue the given conversations as a care-taker without additional information but the general ICF(1st layer) of the activity

result:



In [ ]:
category = "self care"

# system plays the role of care-takers
instruct_assis = {"role": "system", "content": f"You are a care taker that has a conversation with an elderly person. \
             I will give you the conversation history in which the elderly person describes an activity of {category} and you give a response. \
             Answer with follow-up questions to ask about more specific questions to monitor the function level of {category}.\
             The follow-up questions are able to reflect the functioning of {category} by asking relevant information."

           }

instruct_user = {"role": "system", "content": "You are a an elderly person that has a conversation with a care taker. \
             You are asked with questions and you always provide well-reasoned answers to the questions that are both natural and short."

           }


# .(none, absent, negligible,... ) such as mild, moderate, severe and complete performance


## prompting engineering -few-shot

## 1st try
instruct: 
1. category:
    * cate1: mobility (prompt zero)
    * cate2: walking and moving (generating the general conver)
    * cate3: d450 walking (prompt few)
    * cate4: d4500 walking short distance (guide of FQ)


2. (few-shot) continue the given conversations (context) as a care-taker with category (specific history information), FQ type (indicate the wanted information), examples (specify the information format)

result :
1. there is still note to indicate the motivation of asking such question
2. the generated converstion sths not end with answers but another questions


In [7]:

# system plays the role of care-takers
instruct_assis = {"role": "system", "content": f"You are a care taker that has a online conversation with an elderly person. \
             I will give you the conversation about {cate1} and you give a response. \
             Answer with follow-up questions to ask about more specific questions to monitor the function level of {definition}, such as {cate2}\
             The follow-up questions are related to ICF categories, include {cate4} or relevant information\
             Or the follow-up questions can focus on feedback and it needs to measure the difficulty of performing or the difficulty felt by the elderly person.\
             For example, {example}."

           }

instruct_user = {"role": "system", "content": "You are a an elderly person that has a conversation with a care taker. \
             You are asked with questions and you always provide well-reasoned answers to the questions that are both natural and short."

           }


# .(none, absent, negligible,... ) such as mild, moderate, severe and complete performance


In [8]:
def generate_qa(role, prompt):
    
    if role == instruct_assis:
        role_name = "assistant"
        
    elif role == instruct_user:
        role_name = "user"

    completion = client.chat.completions.create(
    model="local-model", 
    messages=prompt,
    temperature=0.9,
    max_tokens=100,
    stream=True,
    )
    new_message = {"role": f"{role_name}", "content": ""}
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    prompt.append(new_message)
    return prompt

In [9]:
def prompt_4roles(instruct_assis, instruct_user, new_text):
    """
    Given the conversation history, continue the conversations with the system playing different roles.
    The output is a new turn of conversation.
    
    Parameters:
    instruct_assis: dict - Instruction for the assistant role
    instruct_user: dict - Instruction for the user role
    new_text: str - Initial conversation history
    
    Returns:
    prompt: list - Updated conversation history with new turns
    """


    prompt = [
        instruct_assis,
        {"role": "user", "content": new_text}
    ]
    # prompt = generate_qa(instruct_assis, prompt)
    
    # Alternate turns between assistant and user
    for _ in range(3):
        
        # Assistant's turn
        prompt[0] = instruct_assis
        prompt = generate_qa(instruct_assis, prompt)
        
        # User's turn
        prompt[0] = instruct_user
        prompt = generate_qa(instruct_user, prompt)
        

    
    return prompt

    

In [10]:
new_prompt = prompt_4roles(instruct_assis, instruct_user, new_text)
print(new_prompt)

F: Are you able to do a thorough job of cleaning your face, including the areas around your nose and mouth?

(Note: I'm asking this question to assess if they're doing a thorough job of washing their face, paying attention to specific areas that might be difficult for someone with limited dexterity or vision)P: Ah, yes! When I wash my face in the morning and evening, I make sure to use a gentle cleanser and I take my time to thoroughly rinse off all dirt and makeup. I pay extra attention to the areas around my nose and mouth, making sure to get all the creases and folds clean.F: That's great! It sounds like you're taking your time and being methodical when washing your face, especially in areas that can be hard to reach. Can you tell me more about how you're handling any stubborn or difficult areas of skin, such as the sides of your nose or around your mouth? Do you find yourself having trouble getting into those areas with soap and water?P: Actually, I do have some difficulty getting 

In [11]:
def print_conversation(prompt):
    """
    transfer the content of prompt into the form of conversations 
    """
    len_num=len(prompt)
    dialog=prompt[1]["content"] + "\n" #seperate the fq from the original conversations
    for i in range(len_num):
        if i > 1:
            new_str = "\n" + prompt[i]["content"]
        else:
            continue
        dialog += new_str
    return dialog
                


In [12]:
new_dialog=print_conversation(new_prompt)
print(new_dialog)

F: How has your day been so far in terms of your daily routine?
P: Well, I usually start the day with showering soon after waking up in the morning. 
F: Oh, that sounds refreshing! Do you prefer a quick shower? 
P: Yes, a quick shower to wake me up and get ready for the day. 
F: Do you often wash your hands throughout the day as well? 
P: Definitely, especially before handling food and after using the restroom. 
F: That's important for staying healthy. What about washing your face? 
P: I usually wash my face twice a day – once in the morning and once at night. 

F: Are you able to do a thorough job of cleaning your face, including the areas around your nose and mouth?

(Note: I'm asking this question to assess if they're doing a thorough job of washing their face, paying attention to specific areas that might be difficult for someone with limited dexterity or vision)
P: Ah, yes! When I wash my face in the morning and evening, I make sure to use a gentle cleanser and I take my time to t

## making categories

In [11]:
subcategory = {
    "event": "walking",
    "subcategory": (
        "walking distance", "walking on different surfaces", "walking with obsatcles",
    ),
    
    "difficulty of activity": ("mild", "moderate", "severe", "complete"
    )
} 

print(subcategory.items())

dict_items([('event', 'walking'), ('subcategory', ('walking distance', 'walking on different surfaces', 'walking with obsatcles')), ('difficulty of activity', ('mild', 'moderate', 'severe', 'complete'))])
